# Store data

As calling API everytime may takes a while and we may hit the limit of API, we will store the data from API for now. 

In [1]:
import tmdbsimple as tmdb
tmdb.API_KEY = "71e259894a515060876bab2a33d6bdc9"

In [2]:
from imdb import IMDb
import pandas as pd
from PIL import Image
from StringIO import StringIO
import requests
import os
import time
from shutil import copyfile

In [3]:
dir_python_notebook = os.getcwd()
dir_movie_project = os.path.abspath(os.path.join(dir_python_notebook, os.pardir))
dir_data = os.path.join(dir_movie_project, 'data')

# TMDB

In [22]:
# get column names to retrieve information
tmdb_movie_id = tmdb.Movies(2)
movie_info = tmdb_movie_id.info()
tmdb_info_column  = tmdb_movie_id.info().keys()
tmdb_info_column  = [str(c) for c in tmdb_info_column]
tmdb_info_column.sort()

In [23]:
tmdb_info_column

['adult',
 'backdrop_path',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [29]:
movies = pd.DataFrame(columns=tmdb_info_column)
tmdb_filename = str(dir_data)+'\\drv_tmdb_movie_details.json'
tmdb_filename_backup = str(dir_data)+'\\drv_tmdb_movie_details_bkp.json'

# if we already have a movie data file, we can just continue appending it instead of starting from id 1 again.
i_start = 1
i_end = i_start+10000
i_list = range(i_start,i_end)

if os.path.isfile(tmdb_filename):
    movies = pd.read_json(tmdb_filename)
    if len(movies['id']) >0:        
        movie_ids = movies['id'].tolist()
        tmdb_info_column = movies.columns
        i_list = [x for x in i_list if x not in movie_ids]
    else:
        movies = pd.DataFrame(columns=tmdb_info_column)
        

for i in i_list:
    #skip the non-existing movie ids
    if (i % 40 == 0):
        # make sure we do not hit API limit
        time.sleep(12) 
    if (i % 100 == 0):
        movies.to_json(path_or_buf= tmdb_filename)
    try:
        tmdb_movies = tmdb.Movies(i)
        info = tmdb_movies.info()
        
        movie_details = []
        for c in tmdb_info_column:
            if info.has_key(c):
                if info[c] is not None:
                    if c in ['genres','spoken_languages','production_countries','production_companies']:
                        movie_details.append([d['name'] for d in info[c]])
                    elif c in ['belongs_to_collection']:
                        movie_details.append(info[c]['name'])
                    else:
                        movie_details.append(info[c]) 
                else:
                    movie_details.append(None)
            else:
                    movie_details.append(None)
        movies.loc[i] = movie_details
    except Exception:
        continue
        
movies.to_json(path_or_buf= tmdb_filename)

In [27]:
#make a backup in case of corruption
copyfile(tmdb_filename, tmdb_filename_backup)

In [25]:
tmdb_filename = str(dir_data)+'\\drv_tmdb_movie_details.json'
movies = pd.read_json(tmdb_filename)

In [28]:
movies.head(5)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
10003,False,/lJnl8xIhplfzUBtlJLsWBwvXVBj.jpg,The Saint Collection,68000000,"[Thriller, Action, Romance, Science Fiction, A...",,10003,tt0120053,en,The Saint,...,1997-04-03,118063304,116,"[English, Pусский]",Released,Never reveal your name. Never turn your back. ...,The Saint,False,5.8,235
10005,False,/u7IzK6tISpsSuNkWMowl17gSA4e.jpg,Behind Enemy Lines Collection,0,"[Action, Adventure, Thriller]",http://www.foxhome.com/behindenemylinesiiaxiso...,10005,tt0497329,en,Behind Enemy Lines II: Axis of Evil,...,2006-10-17,0,96,"[English, Pусский]",Released,The fate of the world hangs in the balance in ...,Behind Enemy Lines II: Axis of Evil,False,4.6,27
10007,False,/l9fdcaqaQQLE2K8cYwfoIPXi0i7.jpg,See No Evil Collection,8000000,"[Horror, Thriller]",,10007,tt0437179,en,See No Evil,...,2006-05-19,0,84,[English],Released,This summer. Evil gets raw.,See No Evil,False,4.8,105
10009,False,/fAzT4AboZXP2Sj3zE2HcQ7qjMi.jpg,Brother Bear Collection,100000000,"[Adventure, Animation, Family]",http://movies.disney.com/brother-bear,10009,tt0328880,en,Brother Bear,...,2003-10-23,0,85,"[Hrvatski, English, ]",Released,Nature Calls,Brother Bear,False,6.8,964
10010,False,/zhMiJmPuX2o2bbXWWZNUgVbw3OM.jpg,Brother Bear Collection,0,"[Adventure, Animation, Family]",http://movies.disney.com/brother-bear-2,10010,tt0465925,en,Brother Bear 2,...,2006-08-17,0,73,[English],Released,,Brother Bear 2,False,6.3,202
